<a href="https://colab.research.google.com/github/shorey/nlp_deeplearning/blob/master/CBOW_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
np.random.seed(13)
import tensorflow as tf



In [0]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

import gensim

In [4]:

path = get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
corpus = open(path).readlines()[:300]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2

180224/173595 [===============================] - 0s 2us/step


In [0]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    while True:
      for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
          contexts = []
          labels   = []            
          s = index - window_size
          e = index + window_size + 1
            
          contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
          labels.append(word)

          x = sequence.pad_sequences(contexts, maxlen=maxlen)
          y = to_categorical(labels, V)
          yield (x, y)

In [6]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [17]:
ddnb_samples//

2787

In [8]:
cbow.fit_generator(generate_data(corpus,window_size,V), epochs=10,steps_per_epoch=2780)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2780/2780 [==============================] - 12s 4ms/step - loss: 6.2467
Epoch 2/10
2780/2780 [==============================] - 9s 3ms/step - loss: 5.7969
Epoch 3/10
2780/2780 [==============================] - 9s 3ms/step - loss: 5.7497
Epoch 4/10
2780/2780 [==============================] - 9s 3ms/step - loss: 5.7198
Epoch 5/10
2780/2780 [==============================] - 9s 3ms/step - loss: 5.6792
Epoch 6/10
2780/2780 [==============================] - 9s 3ms/step - loss: 5.6443
Epoch 7/10
2780/2780 [==============================] - 9s 3ms/step - loss: 5.6089
Epoch 8/10
2780/2780 [==============================] - 9s 3ms/step - loss: 5.5778
Epoch 9/10
2780/2780 [==============================] - 10s 4ms/step - loss: 5.5472
Epoch 10/10
2780/2780 [==============================] - 9s 3ms/step - loss: 5.5176


In [0]:
vectors = cbow.get_weights()[0]

In [10]:
vectors.shape

(765, 100)

In [11]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

8

In [0]:

vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [0]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [14]:
w2v.most_similar(positive=['the'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('one', 0.6464000940322876),
 ('this', 0.6048001050949097),
 ('a', 0.566098690032959),
 ('out', 0.563056230545044),
 ('any', 0.5628354549407959),
 ('no', 0.5409666299819946),
 ('cats', 0.5340847969055176),
 ('new', 0.5198974609375),
 ('they', 0.5140546560287476),
 ('wonder', 0.5091328620910645)]

In [15]:

w2v.most_similar(positive=['alice'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('she', 0.6458778381347656),
 ('you', 0.5985100269317627),
 ('know', 0.5911593437194824),
 ('several', 0.5889590978622437),
 ('eat', 0.5759929418563843),
 ('ever', 0.5722423195838928),
 ('them', 0.5722321271896362),
 ('so', 0.5650628805160522),
 ('go', 0.5644296407699585),
 ('poor', 0.5603344440460205)]

In [16]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from tensorflow.python.client import device_lib

In [20]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13710784654160871340, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2601785085029051029
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4220580298399739937
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14800692839
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11777016818317079068
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [21]:
14800692839/1024/1024

14115.040625572205